# Модель attr_04
### модель с изменяемой значимостью фичей (различий атрибутов)
### для каждой категории 3-го уровня - своя модель LGBM, class='balanced'

<br><br><br>
# Моделирование / Submit

In [1]:
# SUBMIT 1: Готовим сабмит, используем тестовую выборку Озона
# SUBMIT 0: Тестируем модель на валидационной выборке (Сплитим обучающую выборку Озона на train и test)

SUBMIT = 1

<br><br><br>
# Названия файлов и путей

In [2]:
# 4COLAB: подключаем гугл-диск
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [3]:
# путь до папки с файлами
# files_path = ''
# files_path = 'datasets/'
files_path = '/content/drive/MyDrive/Colab Notebooks/Ozon/datasets/'

# ----- INPUT FILES --------------------------------------------------
# названия файлов обучающей выборки
trn_features_file = 'train_features_02.npz'
trn_other_file = 'train_other_02.parquet'

# названия файлов тестовой выборки
tst_features_file = 'test_features_02.npz'
tst_other_file = 'test_other_02.parquet'

# ----- OUTPUT FILES -------------------------------------------------
# названия выходных файлов
submit_file = 'submit_vova_04.csv'  # модель attr_04 с коэф-ми значимости

<br><br><br>
# Подготовка к работе

In [4]:
# импорт необходимых модулей
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.sparse as sp
from scipy.sparse import csr_matrix

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, precision_recall_curve, auc

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

import warnings

In [5]:
# предупреждения OFF
warnings.filterwarnings('ignore')

In [6]:
# настройки вывода датафреймов
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 50)

In [7]:
# константы для фиксации сидов и разделения на выборки
RND = 42
TEST_SIZE = 0.2

<br><br><br>
# Загрузка файлов, подготовка X_train, X_test, y_train, y_test

In [8]:
%%time
# загрузка файлов обучающей выборки Озона
train = pd.read_parquet(files_path + trn_other_file)
train_spm = sp.load_npz(files_path + trn_features_file)

CPU times: user 5.56 s, sys: 2.58 s, total: 8.13 s
Wall time: 10.4 s


In [9]:
%%time
if SUBMIT:
    # загрузка файлов тестовой выборки Озона
    test = pd.read_parquet(files_path + tst_other_file)
    test_spm = sp.load_npz(files_path + tst_features_file)
    X_train = train_spm
    X_test = test_spm
    y_train = train['target']
else:
    # создание обучающей и валидационной (тестовой) выборки из обучающей выборки Озона
    train, test = train_test_split(train, test_size=TEST_SIZE, random_state=RND, 
                                   stratify=train[['cat3_grouped', 'target']])
    test_spm = train_spm
    X_train = train_spm[train.index]
    X_test = test_spm[test.index]
    y_train = train['target']
    y_test = test['target']

CPU times: user 206 ms, sys: 114 ms, total: 320 ms
Wall time: 973 ms


<br><br><br>
# Функции вычисления и вывода метрик

In [10]:
# переменная-выключатель вывода метрик по категориям (1 - выводить, 0 - нет)
show_cat = 0

In [11]:
# вычисление PR-AUC (исправленная, 3 версия)
def pr_auc_macro(
    target_df: pd.DataFrame,
    predictions_df: pd.DataFrame,
    prec_level: float = 0.75,
    cat_column: str = "cat3_grouped"
) -> float:

    df = target_df.merge(predictions_df, on=["variantid1", "variantid2"])

    y_true = df["target"]
    y_pred = df["scores"]
    categories = df[cat_column]

    weights = []
    pr_aucs = []

    unique_cats, counts = np.unique(categories, return_counts=True)

    if show_cat:
        print()
        print('По категориям:')

    # calculate metric for each big category
    for i, category in enumerate(unique_cats):
        # take just a certain category
        cat_idx = np.where(categories == category)[0]
        y_pred_cat = y_pred[cat_idx]
        y_true_cat = y_true[cat_idx]

        # if there is no matches in the category then PRAUC=0
        if sum(y_true_cat) == 0:
            pr_aucs.append(0)
            weights.append(counts[i] / len(categories))
            if show_cat:
                print(f"---------------   Вес: {counts[i]/len(categories):.5f}   {category}")
            continue
        
        # get coordinates (x, y) for (recall, precision) of PR-curve
        y, x, _ = precision_recall_curve(y_true_cat, y_pred_cat)
        
        # reverse the lists so that x's are in ascending order (left to right)
        y = y[::-1]
        x = x[::-1]
        
        # get indices for x-coordinate (recall) where y-coordinate (precision) 
        # is higher than precision level (75% for our task)
        good_idx = np.where(y >= prec_level)[0]
        
        # if there are more than one such x's (at least one is always there, 
        # it's x=0 (recall=0)) we get a grid from x=0, to the rightest x 
        # with acceptable precision
        if len(good_idx) > 1:
            gt_prec_level_idx = np.arange(0, good_idx[-1] + 1)
        # if there is only one such x, then we have zeros in the top scores 
        # and the curve simply goes down sharply at x=0 and does not rise 
        # above the required precision: PRAUC=0
        else:
            pr_aucs.append(0)
            weights.append(counts[i] / len(categories))
            if show_cat:
                print(f"---------------   Вес: {counts[i]/len(categories):.5f}   {category}")
            continue
        
        # calculate category weight anyway
        weights.append(counts[i] / len(categories))
        # calculate PRAUC for all points where the rightest x 
        # still has required precision 
        try:
            pr_auc_prec_level = auc(x[gt_prec_level_idx], y[gt_prec_level_idx])
            if not np.isnan(pr_auc_prec_level):
                pr_aucs.append(pr_auc_prec_level)
                if show_cat:
                    print(f"pr-auc: {pr_auc_prec_level:.5f}   Вес: {counts[i]/len(categories):.5f}   {category}")
        except ValueError:
            pr_aucs.append(0)
            if show_cat:
                print(f"---------------   Вес: {counts[i]/len(categories):.5f}   {category}")
    if show_cat:
        print()

    return np.average(pr_aucs, weights=weights)

In [12]:
# подготовка инпутов для вычисления, вычисление и возврат значения PR-AUC
def pr_auc_score(test, y_prob):
    target_df = test[['target', 'variantid1', 'variantid2']]
    predictions_df = test[['variantid1', 'variantid2', 'cat3_grouped']]
    predictions_df['scores'] = y_prob
    return pr_auc_macro(target_df, predictions_df, prec_level=0.75, cat_column="cat3_grouped")

In [13]:
def show_results(test, y_pred, y_prob):
    if SUBMIT:
        return
    y_test = test['target']
    print(f'Метрики:')
    print('-'*19)
    print(f"{'Precision:':11s} {precision_score(y_test, y_pred):.5f}")
    print(f"{'Recall:':11s} {recall_score(y_test, y_pred):.5f}")
    print(f"{'F1:':11s} {f1_score(y_test, y_pred):.5f}")
    print(f"{'PR-AUC:':11s} {pr_auc_score(test, y_prob):.5f}")

<br><br><br>
# Моделирование

In [14]:
%%time
# модель 04 (attr_04)    pr-auc 0.47887    submit04 0.08596
# модель с изменяемой значимостью фичей (различий атрибутов)
# для каждой категории своя модель LGBM, class='balanced'


# функция вычисления вектора с количествами значений фичей по столбцам в спарс-матрице
def get_count_vec(matrix, feature_value):
    if feature_value == 0:
        # подсчет вектора с количеством нулей по столбцам
        counts_not_null = matrix.getnnz(axis=0)
        count_vec = matrix.shape[0] - counts_not_null
    else:
        # матрица, как входная matrix, но единички на месте feature_value, остальные нули
        ones_matrix = csr_matrix(((matrix.data == feature_value).astype(int), matrix.indices, matrix.indptr), 
                                 shape=matrix.shape)
        # подсчет единичек по столбцам, результат - вектор с количествами
        count_vec = np.array(ones_matrix.sum(axis=0)).flatten()
    return count_vec


# создание общих векторов предиктов
y_pred = pd.Series(0, index=test.index)
y_prob = pd.Series(0, index=test.index)

# цикл по категориям тестовой выборки
for cat in test['cat3_grouped'].unique():

    # выборки датафреймов для выбранной категории cat
    trn_cat = train[train['cat3_grouped'] == (cat if cat in train['cat3_grouped'].unique() else 'rest')]
    tst_cat = test[test['cat3_grouped'] == cat]
    print(f"{'' if cat in train['cat3_grouped'].unique() else '----- fit on rest ----- '}{cat}", 
          f"{trn_cat.shape} {tst_cat.shape}")

    # выборки спарс-матриц фичей для выбранной категории cat (абсолютное индексирование, от входных матриц)
    X_train_cat = train_spm[trn_cat.index]
    X_test_cat = test_spm[tst_cat.index]
    y_train_cat = trn_cat['target']

    # выборка датафрейма для выбранной категории cat с таргетом 1 (t1) и таргетом 0 (t0)
    trn_cat_t1 = train[(train['target'] == 1) & 
                       (train['cat3_grouped'] == (cat if cat in train['cat3_grouped'].unique() else 'rest'))]
    trn_cat_t0 = train[(train['target'] == 0) & 
                       (train['cat3_grouped'] == (cat if cat in train['cat3_grouped'].unique() else 'rest'))]
    X_train_cat_t1 = train_spm[trn_cat_t1.index]
    X_train_cat_t0 = train_spm[trn_cat_t0.index]

    # для всех кат., кроме запч., меняем значения совпадений (0.1 --> 0.024) и частичных совпадений (0.5 --> 1.1)
    if cat not in ['Запчасти для ноутбуков']:
        X_train_cat = csr_matrix(([v if v >= 1 else 0.024 if v == 0.1 else 1.1 for v in X_train_cat.data], 
                                  X_train_cat.indices, X_train_cat.indptr), shape=X_train_cat.shape)
        X_test_cat = csr_matrix(([v if v >= 1 else 0.024 if v == 0.1 else 1.1 for v in X_test_cat.data], 
                                 X_test_cat.indices, X_test_cat.indptr), shape=X_test_cat.shape)
        X_train_cat_t1 = csr_matrix(([v if v >= 1 else 0.024 if v == 0.1 else 1.1 for v in X_train_cat_t1.data], 
                                     X_train_cat_t1.indices, X_train_cat_t1.indptr), shape=X_train_cat_t1.shape)
        X_train_cat_t0 = csr_matrix(([v if v >= 1 else 0.024 if v == 0.1 else 1.1 for v in X_train_cat_t0.data], 
                                     X_train_cat_t0.indices, X_train_cat_t0.indptr), shape=X_train_cat_t0.shape)

    # обозначения признаков и их значения в матрице фичей v.02
    diff = 1    # (difference)
    mtch = 0.1  # (match)
    part = 0.5  # (partial match)
    ndat = 0    # (no data)

    # вычисление количества всех различий (diff) в категории, плюс всех различий по классам
    counts_diff = get_count_vec(X_train_cat, diff)
    counts_diff_t1 = get_count_vec(X_train_cat_t1, diff)
    counts_diff_t0 = get_count_vec(X_train_cat_t0, diff)

    # множитель для вектора фичей пары
    factor = 1
    if cat in ['Запчасти для ноутбуков', 'Компьютер', 'Запчасти для смартфонов']:
        factor = counts_diff + counts_diff_t0 - counts_diff_t1

    # умножаем поэлементно каждую строку (вектор) матрицы фичей на строку (вектор) factor
    X_train_cat = X_train_cat.multiply(factor)
    X_test_cat = X_test_cat.multiply(factor)

    # обучение и предикты модели для категории
    model = LGBMClassifier(random_state=RND, n_estimators = 500, class_weight = 'balanced', max_depth = 15)
    model.fit(X_train_cat, y_train_cat)
    pred = model.predict(X_test_cat)
    prob = model.predict_proba(X_test_cat)[:,1]

    # добавление предиктов для категории в общие вектора предиктов
    y_pred.loc[tst_cat.index] = pred
    y_prob.loc[tst_cat.index] = prob

Батарейки и аккумуляторы (7656, 6) (596, 5)
Смартфоны, планшеты, мобильные телефоны (42622, 6) (925, 5)
Кабели и переходники (19979, 6) (1078, 5)
Устройство ручного ввода (8517, 6) (413, 5)
Аксессуары для фото и видеотехники (1716, 6) (186, 5)
Расходник для печати (22800, 6) (1564, 5)
Запчасти для смартфонов (6025, 6) (1007, 5)
Запчасти для ноутбуков (6568, 6) (2323, 5)
Видеокарты и графические ускорители (2844, 6) (104, 5)
Смарт-часы (12517, 6) (1545, 5)
Корпуса для компьютеров (573, 6) (53, 5)
Компьютер (42565, 6) (1043, 5)
Зарядные устройства и док-станции (7187, 6) (1849, 5)
Наушники и гарнитуры (10219, 6) (532, 5)
Видеонаблюдение (6216, 6) (309, 5)
Акустика и колонки (3008, 6) (227, 5)
Электронные модули (3427, 6) (393, 5)
Кронштейн (1471, 6) (171, 5)
Офисная техника (555, 6) (39, 5)
Оптические приборы (3758, 6) (133, 5)
Мониторы и запчасти (2580, 6) (136, 5)
Жесткие диски, SSD и сетевые накопители (6806, 6) (565, 5)
Телевизоры (3413, 6) (163, 5)
Сетевые фильтры, разветвители и уд

In [15]:
# переменная-выключатель вывода метрик по категориям (1 - выводить, 0 - нет)
show_cat = 1

# out
show_results(test, y_pred, y_prob)

<br><br><br>
# Подготовка и сохранение CSV-файла для сабмита

### (данный код выполняется, если SUBMIT != 0)

In [16]:
if SUBMIT:
    # создание датафрейма для сабмита из пары столбцов test + столбец с предсказаниями
    submit_df = test[['variantid1', 'variantid2']]
    submit_df['target'] = y_prob

    # сохранение датафрейма для сабмита в файл
    submit_df.to_csv(files_path + submit_file, index=False)

    # out
    display(submit_df)

,variantid1,variantid2,target
0,52076340,290590137,0.007404
1,64525522,204128919,0.137953
2,77243372,479860557,0.538166
3,86065820,540678372,0.799935
4,91566575,258840506,0.459643
...,...,...,...
18079,666998614,667074522,0.000439
18080,670036240,670048449,0.190727
18081,670284509,684323809,0.852871
18082,692172005,704805270,0.603871
